# 3rd model to train image classifier
> using Keras and Tensorflow

**Import the libraries and prepare the dataset.**

In [1]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers import Activation, Dropout, Flatten, Dense

print(tf.__version__)
image_size = 80

ModuleNotFoundError: No module named 'matplotlib'

In [64]:
import os 
from tensorflow.keras.preprocessing import image
 
import pathlib
def getImagePaths(rootPath):
  data_root = pathlib.Path(rootPath)
  image_paths = list(data_root.glob('*/*'))
  image_paths = [str(path) for path in image_paths if not str(path).endswith('DS_Store')]
  random.shuffle(image_paths)
  return image_paths

#image resize -> PIL -> tensorflow 
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize_images(image, [image_size, image_size])
  image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)


In [65]:
# Prepare the train Dataset
all_image_paths = getImagePaths('./data/train/')
data_root = pathlib.Path('./data/train/')
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
print(label_names)
label_to_index = dict((name, index) for index,name in enumerate(label_names))
print(label_to_index)

all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
number_of_images = len(all_image_labels)
all_image_array = []
for image_path in all_image_paths:
  all_image_array.append(load_and_preprocess_image(image_path))

all_image_tensor = tf.convert_to_tensor(all_image_array, dtype=tf.float32)
train_dataset_reshape = tf.reshape(all_image_tensor, [number_of_images, image_size, image_size, 1])
train_dataset_reshape.shape

['barramundi', 'bream', 'snapper']
{'barramundi': 0, 'snapper': 2, 'bream': 1}


TensorShape([Dimension(98), Dimension(80), Dimension(80), Dimension(1)])

In [66]:
# image_path = all_image_paths[1]
# print(image_path)
# x = load_and_preprocess_image(image_path)
# x.shape
# plt.imshow(load_and_preprocess_image(image_path))
# plt.grid(False)
# plt.show()

**Setup the model and train it**

In [67]:
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(image_size, image_size, 1)),
#     keras.layers.Dense(64, activation=tf.nn.relu),
#     keras.layers.Dense(3, activation=tf.nn.softmax)
# ])

# layer1
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), input_shape=(image_size, image_size, 1)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#layer2
model.add(keras.layers.Conv2D(64, (3, 3)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(3, 3)))
#layer3
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
#layer4
model.add(keras.layers.Dense(3))
model.add(keras.layers.Activation('sigmoid'))

In [68]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [69]:
model.fit(train_dataset_reshape, all_image_labels, epochs=100, steps_per_epoch=1)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.1111 - acc: 0.2755
Epoch 2/100
1/1 [==============================] - 1s 695ms/step - loss: 1.0330 - acc: 0.5204
Epoch 3/100
1/1 [==============================] - 1s 903ms/step - loss: 1.0715 - acc: 0.5102
Epoch 4/100
1/1 [==============================] - 1s 892ms/step - loss: 1.0438 - acc: 0.5204
Epoch 5/100
1/1 [==============================] - 1s 692ms/step - loss: 1.0378 - acc: 0.5204
Epoch 6/100
1/1 [==============================] - 1s 654ms/step - loss: 1.0325 - acc: 0.5204
Epoch 7/100
1/1 [==============================] - 1s 662ms/step - loss: 1.0365 - acc: 0.5204
Epoch 8/100
1/1 [==============================] - 1s 666ms/step - loss: 1.0460 - acc: 0.5000
Epoch 9/100
1/1 [==============================] - 1s 839ms/step - loss: 1.0115 - acc: 0.5306
Epoch 10/100
1/1 [==============================] - 1s 685ms/step - loss: 1.0373 - acc: 0.5306
Epoch 11/100
1/1 [==============================] - 1s 680ms/s

Epoch 88/100
1/1 [==============================] - 1s 724ms/step - loss: 0.3582 - acc: 0.9286
Epoch 89/100
1/1 [==============================] - 1s 710ms/step - loss: 0.3581 - acc: 0.8980
Epoch 90/100
1/1 [==============================] - 1s 709ms/step - loss: 0.3712 - acc: 0.8469
Epoch 91/100
1/1 [==============================] - 1s 729ms/step - loss: 0.3364 - acc: 0.8469
Epoch 92/100
1/1 [==============================] - 1s 736ms/step - loss: 0.3577 - acc: 0.8571
Epoch 93/100
1/1 [==============================] - 1s 743ms/step - loss: 0.3475 - acc: 0.8776
Epoch 94/100
1/1 [==============================] - 1s 709ms/step - loss: 0.3583 - acc: 0.8163
Epoch 95/100
1/1 [==============================] - 1s 702ms/step - loss: 0.3522 - acc: 0.8776
Epoch 96/100
1/1 [==============================] - 1s 744ms/step - loss: 0.2732 - acc: 0.9184
Epoch 97/100
1/1 [==============================] - 1s 761ms/step - loss: 0.2689 - acc: 0.9184
Epoch 98/100
1/1 [==============================] 

Test the model and make use of that.

In [70]:
test_loss, test_acc = model.evaluate(train_dataset_reshape, all_image_labels, steps=1)
print('Test accuracy:', test_acc)

1/1 [==============================] - 1s 1s/step
Test accuracy: 0.9591836929321289


In [71]:
# Prepare the prediction Dataset
all_predict_paths = getImagePaths('./data/test/')
all_predict_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_predict_paths]
number_of_predict = len(all_predict_labels)
all_predict_array = []
for predict_path in all_predict_paths:
  print(predict_path)
  all_predict_array.append(load_and_preprocess_image(predict_path))

all_predict_tensor = tf.convert_to_tensor(all_predict_array, dtype=tf.float32)
predict_dataset_reshape = tf.reshape(all_predict_tensor, [number_of_predict, image_size, image_size, 1])
predict_dataset_reshape.shape

data/test/snapper/mutton-snapper-750x400.jpg
data/test/barramundi/barramundi-fish.jpg
data/test/snapper/snapper.jpg
data/test/bream/BREAM-BLACK-1-940x528.jpg
data/test/bream/bluegill-bream.jpeg
data/test/bream/bream_bl_aa.jpg
data/test/snapper/pg-25-thai-snapper-480x201-1.jpg


TensorShape([Dimension(7), Dimension(80), Dimension(80), Dimension(1)])

In [72]:
predictions = model.predict(predict_dataset_reshape, steps=1)
for item in predictions:
  print(np.argmax(item), end=', ')
print()
print(all_predict_labels)

2, 0, 2, 1, 2, 1, 2, 
[2, 0, 2, 1, 1, 1, 2]


In [74]:
# Save tf.keras model in HDF5 format.
keras_file = "keras_model.h5"
tf.keras.models.save_model(model, keras_file)

# Convert to TensorFlow Lite model.
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

AttributeError: 'module' object has no attribute 'TFLiteConverter'